In [2]:
sensor_dtype = np.dtype([
    ("x", ">f4"),
    ("y", ">f4"),
    ("z", ">f4"),
    ("time", ">i8"),
])


def load_raw_sensor_data(path):
    """Load and process sensor data from binary files"""
    acc_files = sorted([f for f in os.listdir(path) if f.endswith('.bin') and 'accelerometer' in f])
    gyro_files = sorted([f for f in os.listdir(path) if f.endswith('.bin') and 'gyroscope' in f])

    all_acc_data = []
    all_gyro_data = []

    # Load accelerometer data
    for file in acc_files:
        boot_time_nanos = int(file.split("_")[0]) * 1e6
        file_path = os.path.join(path, file)
        acc_data = np.fromfile(file_path, dtype=sensor_dtype)
        first_event_time = acc_data['time'][0]
        corrected_timestamps = ((acc_data['time'] - first_event_time) + boot_time_nanos) / 1e9
        corrected_datetimes = pd.to_datetime(corrected_timestamps, unit='s')
        df = pd.DataFrame(acc_data[["x", "y", "z"]].byteswap().newbyteorder())
        df['time'] = corrected_datetimes
        all_acc_data.append(df)

    # Load gyroscope data
    for file in gyro_files:
        boot_time_nanos = int(file.split("_")[0]) * 1e6
        file_path = os.path.join(path, file)
        gyro_data = np.fromfile(file_path, dtype=sensor_dtype)
        first_event_time = gyro_data['time'][0]
        corrected_timestamps = ((gyro_data['time'] - first_event_time) + boot_time_nanos) / 1e9
        corrected_datetimes = pd.to_datetime(corrected_timestamps, unit='s')
        df = pd.DataFrame(gyro_data[["x", "y", "z"]].byteswap().newbyteorder())
        df['time'] = corrected_datetimes
        all_gyro_data.append(df)

    return pd.concat(all_acc_data), pd.concat(all_gyro_data)


def sync_sensors(acc_data, gyro_data):
    """Synchronize accelerometer and gyroscope data"""
    common_start_time = max(acc_data['time'].min(), gyro_data['time'].min())
    common_end_time = min(acc_data['time'].max(), gyro_data['time'].max())

    acc_synced = acc_data[(acc_data['time'] >= common_start_time) & (acc_data['time'] <= common_end_time)]
    gyro_synced = gyro_data[(gyro_data['time'] >= common_start_time) & (gyro_data['time'] <= common_end_time)]

    return acc_synced.reset_index(drop=True), gyro_synced.reset_index(drop=True)


In [3]:
import numpy as np
import pandas as pd
import os
from scipy import stats, signal
from datetime import timedelta

def analyze_dataset(base_path):
    """Analyze entire dataset and compute aggregate statistics"""
    session_dirs = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
    
    # Initialize lists to store all data for aggregate calculations
    all_acc_stats = []
    all_gyro_stats = []
    all_durations = []
    all_sampling_rates = []
    all_temporal_offsets = []
    all_sample_intervals = []
    all_acc_spectral = {'x': [], 'y': [], 'z': []}
    all_gyro_spectral = {'x': [], 'y': [], 'z': []}
    total_samples = 0
    missing_samples_total = 0
    continuous_intervals_total = 0
    total_intervals = 0
    
    for session_dir in session_dirs:
        session_path = os.path.join(base_path, session_dir)
        
        # Load and sync data
        acc_data, gyro_data = load_raw_sensor_data(session_path)
        acc_synced, gyro_synced = sync_sensors(acc_data, gyro_data)
        
        # Session duration
        duration = (acc_synced['time'].max() - acc_synced['time'].min()).total_seconds()
        all_durations.append(duration)
        
        # Temporal characteristics
        time_diffs = acc_synced['time'].diff()[1:].dt.total_seconds()
        all_sample_intervals.extend(time_diffs * 1000)  # Convert to ms
        all_sampling_rates.append(1 / time_diffs.mean())
        
        # Temporal offset
        offset = (acc_synced['time'] - gyro_synced['time']).dt.total_seconds() * 1000
        all_temporal_offsets.extend(offset)
        
        # Accumulate samples for signal statistics
        for axis in ['x', 'y', 'z']:
            acc_data = acc_synced[axis]
            gyro_data = gyro_synced[axis]
            
            # Signal statistics
            all_acc_stats.append({
                'axis': axis,
                'values': acc_data,
                'range': [acc_data.min(), acc_data.max()],
                'mean': acc_data.mean(),
                'std': acc_data.std(),
                'skewness': stats.skew(acc_data),
                'kurtosis': stats.kurtosis(acc_data),
                'snr': calculate_snr(acc_data)
            })
            
            all_gyro_stats.append({
                'axis': axis,
                'values': gyro_data,
                'range': [gyro_data.min(), gyro_data.max()],
                'mean': gyro_data.mean(),
                'std': gyro_data.std(),
                'skewness': stats.skew(gyro_data),
                'kurtosis': stats.kurtosis(gyro_data),
                'snr': calculate_snr(gyro_data)
            })
            
            # Spectral analysis
            f_acc, psd_acc = signal.welch(acc_data, fs=100)
            f_gyro, psd_gyro = signal.welch(gyro_data, fs=100)
            
            total_power_acc = np.sum(psd_acc)
            total_power_gyro = np.sum(psd_gyro)
            
            # Accelerometer spectral components
            all_acc_spectral[axis].append({
                'low_freq': np.sum(psd_acc[(f_acc >= 0.1) & (f_acc < 3)]) / total_power_acc * 100,
                'mid_freq': np.sum(psd_acc[(f_acc >= 3) & (f_acc < 10)]) / total_power_acc * 100,
                'high_freq': np.sum(psd_acc[f_acc >= 10]) / total_power_acc * 100
            })
            
            # Gyroscope spectral components
            all_gyro_spectral[axis].append({
                'low_freq': np.sum(psd_gyro[(f_gyro >= 0.1) & (f_gyro < 3)]) / total_power_gyro * 100,
                'mid_freq': np.sum(psd_gyro[(f_gyro >= 3) & (f_gyro < 10)]) / total_power_gyro * 100,
                'high_freq': np.sum(psd_gyro[f_gyro >= 10]) / total_power_gyro * 100
            })
        
        # Data quality metrics
        total_samples += len(acc_synced)
        expected_samples = int(duration * 100)  # Assuming 100Hz target rate
        missing_samples_total += expected_samples - len(acc_synced)
        continuous_intervals = sum(time_diffs < 0.012)  # 12ms threshold for continuity
        total_intervals += len(time_diffs)
        
    # Calculate aggregate statistics
    dataset_stats = {
        'overview': {
            'total_sessions': len(session_dirs),
            'total_duration_hours': sum(all_durations) / 3600,
            'avg_session_duration_minutes': np.mean(all_durations) / 60,
            'session_duration_std_minutes': np.std(all_durations) / 60,
            'mean_sampling_rate': np.mean(all_sampling_rates),
            'sampling_rate_std': np.std(all_sampling_rates),
            'total_samples': total_samples
        },
        'temporal': {
            'mean_temporal_offset_ms': np.mean(all_temporal_offsets),
            'temporal_offset_std_ms': np.std(all_temporal_offsets),
            'sample_interval_ci': stats.norm.interval(0.95, 
                                                    loc=np.mean(all_sample_intervals),
                                                    scale=np.std(all_sample_intervals))
        },
        'acc_stats': aggregate_sensor_stats(all_acc_stats),
        'gyro_stats': aggregate_sensor_stats(all_gyro_stats),
        'acc_spectral': aggregate_spectral_stats(all_acc_spectral),
        'gyro_spectral': aggregate_spectral_stats(all_gyro_spectral),
        'data_quality': {
            'missing_samples_pct': (missing_samples_total / (total_samples + missing_samples_total)) * 100,
            'timestamp_continuity': (continuous_intervals / total_intervals) * 100
        }
    }
    
    return dataset_stats

def calculate_snr(signal_data):
    """Calculate Signal-to-Noise Ratio"""
    f, psd = signal.welch(signal_data, fs=100)
    signal_power = np.sum(psd[f < 10])
    noise_power = np.sum(psd[f >= 10])
    return 10 * np.log10(signal_power / noise_power) if noise_power > 0 else 0

def aggregate_sensor_stats(stats_list):
    """Aggregate sensor statistics across all sessions"""
    stats_by_axis = {'x': [], 'y': [], 'z': []}
    for stat in stats_list:
        axis = stat['axis']
        stats_by_axis[axis].append(stat)
    
    aggregated = {}
    for axis in ['x', 'y', 'z']:
        axis_stats = stats_by_axis[axis]
        aggregated[axis] = {
            'range': [
                min(s['range'][0] for s in axis_stats),
                max(s['range'][1] for s in axis_stats)
            ],
            'mean': np.mean([s['mean'] for s in axis_stats]),
            'std': np.mean([s['std'] for s in axis_stats]),
            'skewness': np.mean([s['skewness'] for s in axis_stats]),
            'kurtosis': np.mean([s['kurtosis'] for s in axis_stats]),
            'snr': np.mean([s['snr'] for s in axis_stats])
        }
    return aggregated

def aggregate_spectral_stats(spectral_data):
    """Aggregate spectral statistics across all sessions"""
    aggregated = {}
    for axis in ['x', 'y', 'z']:
        axis_data = spectral_data[axis]
        aggregated[axis] = {
            'low_freq': np.mean([d['low_freq'] for d in axis_data]),
            'mid_freq': np.mean([d['mid_freq'] for d in axis_data]),
            'high_freq': np.mean([d['high_freq'] for d in axis_data])
        }
    return aggregated

# Usage
if __name__ == "__main__":
    dataset_path = "../data/raw"
    stats = analyze_dataset(dataset_path)
    
    print("\nDataset Overview:")
    print(f"Total Sessions: {stats['overview']['total_sessions']}")
    print(f"Total Recording Duration: {stats['overview']['total_duration_hours']:.1f} hours")
    print(f"Average Session Duration: {stats['overview']['avg_session_duration_minutes']:.1f} minutes")
    print(f"Session Duration (std): {stats['overview']['session_duration_std_minutes']:.1f} minutes")
    print(f"Sampling Rate: {stats['overview']['mean_sampling_rate']:.2f} Hz")
    print(f"Total Samples: {stats['overview']['total_samples']:,}")
    
    print("\nTemporal Characteristics:")
    print(f"Mean Sampling Rate: {stats['overview']['mean_sampling_rate']:.2f} Hz (σ = {stats['overview']['sampling_rate_std']:.2f} Hz)")
    print(f"Mean Temporal Offset: {stats['temporal']['mean_temporal_offset_ms']:.1f} ms (σ = {stats['temporal']['temporal_offset_std_ms']:.1f} ms)")
    
    print("\nAccelerometer Signal Statistics:")
    for axis in ['x', 'y', 'z']:
        print(f"\n{axis}-axis:")
        acc_stats = stats['acc_stats'][axis]
        print(f"Range: [{acc_stats['range'][0]:.1f}, {acc_stats['range'][1]:.1f}] m/s²")
        print(f"Mean: {acc_stats['mean']:.2f} m/s²")
        print(f"Standard Deviation: {acc_stats['std']:.2f}")
        print(f"Skewness: {acc_stats['skewness']:.2f}")
        print(f"Kurtosis: {acc_stats['kurtosis']:.2f}")
        print(f"SNR: {acc_stats['snr']:.1f} dB")
    
    print("\nGyroscope Signal Statistics:")
    for axis in ['x', 'y', 'z']:
        print(f"\n{axis}-axis:")
        gyro_stats = stats['gyro_stats'][axis]
        print(f"Range: [{gyro_stats['range'][0]:.1f}, {gyro_stats['range'][1]:.1f}] rad/s")
        print(f"Mean: {gyro_stats['mean']:.2f} rad/s")
        print(f"Standard Deviation: {gyro_stats['std']:.2f}")
        print(f"Skewness: {gyro_stats['skewness']:.2f}")
        print(f"Kurtosis: {gyro_stats['kurtosis']:.2f}")
        print(f"SNR: {gyro_stats['snr']:.1f} dB")
    
    print("\nSpectral Characteristics:")
    print("Accelerometer frequency components:")
    for axis in ['x', 'y', 'z']:
        acc_spectral = stats['acc_spectral'][axis]
        print(f"\nSpectral components for {axis}-axis:")
        print(f"Low-frequency (0.1-3 Hz): {acc_spectral['low_freq']:.1f}%")
        print(f"Mid-frequency (3-10 Hz): {acc_spectral['mid_freq']:.1f}%")
        print(f"High-frequency (>10 Hz): {acc_spectral['high_freq']:.1f}%")
    
    print("\nGyroscope frequency components:")
    for axis in ['x', 'y', 'z']:
        gyro_spectral = stats['gyro_spectral'][axis]
        print(f"\nSpectral components for {axis}-axis:")
        print(f"Low-frequency (0.1-3 Hz): {gyro_spectral['low_freq']:.1f}%")
        print(f"Mid-frequency (3-10 Hz): {gyro_spectral['mid_freq']:.1f}%")
        print(f"High-frequency (>10 Hz): {gyro_spectral['high_freq']:.1f}%")
    
    print("\nData Quality Metrics:")
    print(f"Missing Samples: {stats['data_quality']['missing_samples_pct']:.2f}%")
    print(f"Timestamp Continuity: {stats['data_quality']['timestamp_continuity']:.2f}%")
    print(f"Sample Interval 95% CI: [{stats['temporal']['sample_interval_ci'][0]:.1f}, {stats['temporal']['sample_interval_ci'][1]:.1f}] ms")

print("\nNote: The following metrics cannot be computed from raw sensor data alone:")
print("- Number of unique subjects")
print("- Active eating duration and non-eating duration")
print("- Movement pattern analysis specific to eating gestures")
print("These would require additional metadata or annotations.")


Dataset Overview:
Total Sessions: 26
Total Recording Duration: 9.6 hours
Average Session Duration: 22.2 minutes
Session Duration (std): 10.9 minutes
Sampling Rate: 51.84 Hz
Total Samples: 1,800,416

Temporal Characteristics:
Mean Sampling Rate: 51.84 Hz (σ = 1.08 Hz)
Mean Temporal Offset: nan ms (σ = nan ms)

Accelerometer Signal Statistics:

x-axis:
Range: [-37.2, 36.4] m/s²
Mean: -0.75 m/s²
Standard Deviation: 3.76
Skewness: 0.28
Kurtosis: 1.40
SNR: 15.4 dB

y-axis:
Range: [-49.1, 35.8] m/s²
Mean: -6.37 m/s²
Standard Deviation: 3.18
Skewness: 1.07
Kurtosis: 2.19
SNR: 11.7 dB

z-axis:
Range: [-34.4, 54.6] m/s²
Mean: 2.83 m/s²
Standard Deviation: 4.16
Skewness: -0.06
Kurtosis: -0.40
SNR: 15.1 dB

Gyroscope Signal Statistics:

x-axis:
Range: [-30.1, 30.0] rad/s
Mean: -0.00 rad/s
Standard Deviation: 0.91
Skewness: 0.37
Kurtosis: 26.98
SNR: 12.2 dB

y-axis:
Range: [-30.0, 29.4] rad/s
Mean: 0.00 rad/s
Standard Deviation: 0.44
Skewness: -0.01
Kurtosis: 56.48
SNR: 15.5 dB

z-axis:
Range: [-